In [1]:
import tensorflow as tf
import pickle
import os
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Activation, Conv2D, Conv2DTranspose, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.layers import ReLU, Resizing, Rescaling, concatenate
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.summary import scalar, create_file_writer
import time
import datetime
import keras
from tensorflow.io import read_file, decode_jpeg
from tensorflow import cast, shape
from tensorflow.image import resize_with_pad, ResizeMethod

# Generator and Discriminator

In [2]:
def downsample(filters, size, strides, apply_batchnorm = True):
    initializer = tf.random_normal_initializer(0., 0.02)
    
    # Make sequential model
    result = Sequential()
    # Add Conv2D
    result.add(Conv2D(filters, size, strides = strides, padding = 'same',
                             kernel_initializer = initializer, use_bias = False))

    # Optionally add batchnorm
    if apply_batchnorm:
        result.add(BatchNormalization())

    # Add leaky relu
    result.add(LeakyReLU(alpha = 0.2))

    return result

In [3]:
def upsample(filters, size, strides, apply_dropout = False):
    initializer = tf.random_normal_initializer(0., 0.02)

    # Make sequential model
    result = Sequential()
    # Add deconv layer (conv2dtranspose)
    result.add(Conv2DTranspose(filters, size, strides = strides,
                                    padding='same',
                                    kernel_initializer = initializer,
                                    use_bias = False))

    # Add batchnorm
    result.add(BatchNormalization())

    # Optionally add dropout
    if apply_dropout:
        result.add(Dropout(0.5))
    
    # Add relu
    result.add(ReLU())

    return result

In [4]:
def create_generator():
    inputs = Input(shape = [256, 256, 1])
    
    # Define downsampling layers
    down_stack = [
        downsample(64, 4, 2, apply_batchnorm = False),
        downsample(128, 4, 2),
        downsample(256, 4, 2),
        downsample(512, 4, 2),
        downsample(512, 4, 2),
        downsample(512, 4, 2),
        downsample(512, 4, 2),
        downsample(512, 4, 2)
    ]
    
    # Define upsampling layers
    up_stack = [
        upsample(512, 4, 2, apply_dropout = True),
        upsample(512, 4, 2, apply_dropout = True),
        upsample(512, 4, 2, apply_dropout = True),
        upsample(512, 4, 2),
        upsample(256, 4, 2),
        upsample(128, 4, 2),
        upsample(64, 4, 2),
    ]
    
    # Last layer
    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(1, 4, strides = 2, padding = 'same',
                                           kernel_initializer = initializer, activation = 'tanh')
    
    x = inputs

    # Downsampling through the model
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])

    x = last(x)

    return tf.keras.Model(inputs = inputs, outputs = x)

In [5]:
def create_discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = tf.keras.layers.Input(shape = [256, 256, 1], name = 'sketch')
    tar = tf.keras.layers.Input(shape = [256, 256, 1], name = 'target')

    x = tf.keras.layers.concatenate([inp, tar])

    down1 = downsample(64, 4, 2, False)(x)
    down2 = downsample(128, 4, 2)(down1)
    down3 = downsample(256, 4, 2)(down2)
    down4 = downsample(512, 4, 1)(down3)

    last = tf.keras.layers.Conv2D(1, 4, strides = 1, kernel_initializer = initializer)(down4)

    return tf.keras.Model(inputs = [inp, tar], outputs = last)

# Classes and Losses

In [7]:
# log_dir="logs/"

# summary_writer = tf.summary.create_file_writer(
#   log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [8]:
# def fit(train_ds, test_ds, steps):
#     start = time.time()

#     for step, (input_image, target) in train_ds.repeat().take(steps).enumerate():
#         if (step) % 1000 == 0:
#             display.clear_output(wait=True)

#     if step != 0:
#         print(f'Time taken for 1000 steps: {time.time()-start:.2f} sec\n')

#     start = time.time()

#   print(f"Step: {step//1000}k")
# train_step(input_image, target, step)

# # Training step
# if (step+1) % 10 == 0:
#   print('.', end='', flush=True)


# # Save (checkpoint) the model every 5k steps
# if (step + 1) % 5000 == 0:
#   checkpoint.save(file_prefix=checkpoint_prefix)

In [9]:
#     def train_step(self, sketches, images):
                
#         with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
#             generated_images = self.generator(sketches, training = True)

#             predict_real = self.discriminator([sketches, images], training = True)
#             predict_fake = self.discriminator([sketches, generated_images], training = True)

#             gen_step_loss = self.g_loss(images, generated_images, predict_fake, self.net1, self.net2)
#             disc_step_loss = self.d_loss(predict_real, predict_fake)

#         gen_grad = gen_tape.gradient(gen_step_loss, self.generator.trainable_variables)
#         disc_grad = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

#         generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
#         discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        
#         # Monitor loss.
#         self.gen_loss_tracker.update_state(gen_step_loss)
#         self.disc_loss_tracker.update_state(disc_step_loss)

#         return {"g_loss": self.gen_loss_tracker.result(), "d_loss": self.disc_loss_tracker.result()}

# Preprocessing

In [10]:
# image_datagen = ImageDataGenerator(
#     rescale = 1./255
# )

In [11]:
# sketch_datagen = ImageDataGenerator(
#     rescale = 1./255
# )

In [12]:
# train_generator_image = image_datagen.flow_from_directory(
#     '../data/train/images',
#     target_size = (256, 256),
#     batch_size = 32,
#     class_mode = None,
#     color_mode = 'grayscale'
# )

# train_generator_sketch = sketch_datagen.flow_from_directory(
#     '../data/train/sketches',
#     target_size = (256, 256),
#     batch_size = 32,
#     class_mode = None,
#     color_mode = 'grayscale'
# )

In [13]:
# test_generator_image = image_datagen.flow_from_directory(
#     '../data/test/images',
#     target_size = (256, 256),
#     batch_size = 32,
#     class_mode = None,
#     color_mode = 'grayscale'
# )

# test_generator_sketch = sketch_datagen.flow_from_directory(
#     '../data/test/sketches',
#     target_size = (256, 256),
#     batch_size = 32,
#     class_mode = None,
#     color_mode = 'grayscale'
# )

In [14]:
# validation_generator_image = image_datagen.flow_from_directory(
#     '../data/train/images',
#     target_size=(256, 256),
#     batch_size=32,
#     class_mode=None,
#     color_mode = 'grayscale',
#     subset='validation') # set as validation data

# validation_generator_sketch = sketch_datagen.flow_from_directory(
#     '../data/train/sketches',
#     target_size=(256, 256),
#     batch_size=32,
#     class_mode=None,
#     color_mode = 'grayscale',
#     subset='validation') # set as validation data

In [15]:
def load(image_file):
  # Read and decode an image file to a uint8 tensor
    pair = read_file(image_file)
    pair = decode_jpeg(pair)
    
    w = shape(pair)[1]
    w = w // 2
    
    image = pair[:, w:, :]
    sketch = pair[:, :w, :]

  # Convert both images to float32 tensors
    sketch = cast(sketch, tf.float32)
    image = cast(image, tf.float32)

    return sketch, image

In [16]:
def resize(sketch, image, height, width):
    sketch_resized = resize_with_pad(sketch, height, width, method = ResizeMethod.NEAREST_NEIGHBOR)
    image_resized = resize_with_pad(image, height, width, method = ResizeMethod.NEAREST_NEIGHBOR)

    return sketch_resized, image_resized

In [17]:
# Normalizing the images to [0, 1]
def normalize(sketch, image):
    sketch_scaled = sketch / 255
    image_scaled = image / 255

    return sketch_scaled, image_scaled

In [18]:
def load_image_and_sketch(image_file):
    sketch, image = load(image_file)
    sketch, image = resize(sketch, image, 256, 256)
    sketch, image = normalize(sketch, image)

    return sketch, image

In [19]:
train_dataset = tf.data.Dataset.list_files('../data/train/*.jpg')
train_dataset = train_dataset.map(load_image_and_sketch,
                                  num_parallel_calls = tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(1524)
train_dataset = train_dataset.batch(32)

In [20]:
validation_dataset = tf.data.Dataset.list_files('../data/validation/*.jpg')
validation_dataset = validation_dataset.map(load_image_and_sketch,
                                  num_parallel_calls = tf.data.AUTOTUNE)
validation_dataset = validation_dataset.batch(32)

In [21]:
test_dataset = tf.data.Dataset.list_files('../data/test/*.jpg')
test_dataset = test_dataset.map(load_image_and_sketch,
                                  num_parallel_calls = tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32)

In [22]:
generator = create_generator()
discriminator = create_discriminator()
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)
net1 = VGG16()
net2 = VGG16()

In [110]:
net1.get_layer('block3_conv3')

In [127]:
net1.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [104]:
net1.block3_conv3

AttributeError: 'Functional' object has no attribute 'block3_conv3'

In [159]:
from PIL import Image

In [161]:
image = Image.open('../data/raw/actual_owls/resized/Owl_Resized_0001.jpg')
net1.build(image)
output = net1.get_layer('block3_conv3').output

In [184]:
tf.reshape(output, (56,56,256))

<KerasTensor: shape=(56, 56, 256) dtype=float32 (created by layer 'tf.reshape')>

In [167]:
image2 = Image.open('../data/raw/actual_owls/sketched/resized/Owl_Sketched_Resized_0001.jpg')
net1.build(image2)
output2 = net1.get_layer('block3_conv3').output

In [185]:
tf.reshape(output, (56,56,256)) - tf.reshape(output2, (56,56,256))

<KerasTensor: shape=(56, 56, 256) dtype=float32 (created by layer 'tf.math.subtract_13')>

In [189]:
# tf.reduce_mean
# tf.sqrt(
tensor = tf.nn.l2_loss(tf.reshape(output, (56,56,256)) - tf.reshape(output2, (56,56,256)))

In [192]:
tensor

<KerasTensor: shape=() dtype=float32 (created by layer 'tf.nn.l2_loss_10')>

In [154]:
model = autopainter(discriminator, generator, net1, net2)

In [155]:
# d_loss = discrim_loss()
# g_loss = gen_loss()

In [156]:
model.compile(discriminator_optimizer, generator_optimizer)

In [157]:
model.fit(train_dataset, epochs = 10, validation_data = validation_dataset)

Epoch 1/10


TypeError: in user code:

    File "/home/joshua/anaconda3/envs/learn-env/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/joshua/anaconda3/envs/learn-env/lib/python3.8/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/joshua/anaconda3/envs/learn-env/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "<ipython-input-153-6de2bcc5c56f>", line 81, in train_step
        gen_total_loss, gen_loss_GAN, gen_loss_L1, gen_loss_tv, gen_loss_f =\
    File "/home/joshua/anaconda3/envs/learn-env/lib/python3.8/site-packages/keras/engine/keras_tensor.py", line 406, in __iter__
        raise TypeError("Cannot iterate over a scalar.")

    TypeError: Cannot iterate over a scalar.


In [153]:
def sum_tv_loss(image):
    loss_y = tf.nn.l2_loss(image[:, 1:, :, :] - image[:, :-1, :, :])
    loss_x = tf.nn.l2_loss(image[:, :, 1:, :] - image[:, :, :-1, :])
    loss = 2 * (loss_y + loss_x)
    loss = tf.cast(loss, tf.float32)
    return loss

def feature_loss(image, vgg):
    vgg.build(image)
    output = vgg.get_layer('block3_conv3').output
#     input_tensor = tf.keras.Input(shape = (56, 56, 256))
#     new_model = tf.keras.Model(inputs = input_tensor, outputs = conv_layer(input_tensor))
#     output = new_model(image)
    
    return output

def discriminator_loss(predict_real, predict_fake):
    return tf.reduce_mean(-(tf.log(predict_real + 1e-12) + tf.log(1 - predict_fake + 1e-12)))

def generator_loss(targets, outputs, predict_fake, net1, net2):
    gen_loss_GAN = -tf.reduce_mean(predict_fake)
    gen_loss_L1 = tf.reduce_mean(tf.abs(targets - outputs))
    gen_loss_tv = tf.reduce_mean(tf.sqrt(tf.nn.l2_loss(sum_tv_loss(outputs))))
    gen_loss_f = tf.reduce_mean(tf.sqrt(tf.nn.l2_loss(feature_loss(targets,net1) - feature_loss(outputs,net2))))
    gen_total_loss = gen_loss_GAN + (gen_loss_L1 * 10) + (gen_loss_tv * 1e-5) + (gen_loss_f * 1e-4)
    
    return gen_total_loss, gen_loss_GAN, gen_loss_L1, gen_loss_tv, gen_loss_f

class discrim_loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
        
    def call(self, y_true, y_pred):
        return descriminator_loss(y_true, y_pred)
    
class gen_loss(tf.keras.losses.Loss):
    def __init__(self, predict_fake = None, net1 = None, net2 = None):
        super().__init__()
        self.predict_fake = predict_fake
        self.net1 = net1
        self.net2 = net2

    def call(self, y_true, y_pred):
        return generator_loss(y_true, y_pred, self.predict_fake, self.net1, self.net2)
    
class autopainter(tf.keras.Model):
    
    def __init__(self, discriminator, generator, net1, net2):
        super(autopainter, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")
        self.net1 = net1
        self.net2 = net2
    
    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]
    
    def compile(self, d_optimizer, g_optimizer):
        super(autopainter, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss = discrim_loss()
        self.g_loss = gen_loss(predict_fake = None, net1 = self.net1, net2 = self.net2)
        self.summary_writer = create_file_writer('../logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
        
    def train_step(self, pair):
        sketch = pair[0]
        target = pair[1]
        
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            gen_output = self.generator(sketch, training = True)

            disc_real_output = self.discriminator([sketch, target], training = True)
            disc_generated_output = self.discriminator([sketch, gen_output], training = True)
            
            self.g_loss.predict_fake = disc_generated_output

            gen_total_loss, gen_loss_GAN, gen_loss_L1, gen_loss_tv, gen_loss_f =\
                                self.g_loss(target, gen_output)
            disc_loss = self.d_loss(disc_real_output, disc_generated_output)

        generator_gradients = gen_tape.gradient(gen_total_loss,
                                              self.generator.trainable_variables)
        discriminator_gradients = disc_tape.gradient(disc_loss,
                                                   self.discriminator.trainable_variables)

        self.g_optimizer.apply_gradients(zip(generator_gradients, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(discriminator_gradients, self.discriminator.trainable_variables))

        with self.summary_writer.as_default():
            scalar('gen_total_loss', gen_total_loss)
#                    , step = step // 1000)
            scalar('gen_gan_loss', gen_loss_GAN)
#                    , step = step // 1000)
            scalar('gen_l1_loss', gen_loss_L1)
#                    , step = step // 1000)
            scalar('gen_tv_loss', gen_loss_tv)
#                    , step = step // 1000)
            scalar('gen_f_loss', gen_loss_f)
#                    , step = step // 1000)
            scalar('disc_loss', disc_loss)
#                    , step = step // 1000)

# Checkpointing and Saving

In [ ]:
checkpoint_dir = '../training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer,
                                 discriminator_optimizer = discriminator_optimizer,
                                 generator = generator,
                                 discriminator = discriminator,
                                 net1 = net1,
                                 net2 = net2)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [ ]:
# def conv(batch_input, out_channels, stride):
#     with tf.variable_scope("conv"):
#         in_channels = batch_input.get_shape()[3]
#         filter = tf.get_variable("filter", [4, 4, in_channels, out_channels], dtype=tf.float32,
#                                  initializer=tf.random_normal_initializer(0, 0.02))
#         # [batch, in_height, in_width, in_channels], [filter_width, filter_height, in_channels, out_channels]
#         #     => [batch, out_height, out_width, out_channels]
#         padded_input = tf.pad(batch_input, [[0, 0], [1, 1], [1, 1], [0, 0]], mode="CONSTANT")
#         conv = tf.nn.conv2d(padded_input, filter, [1, stride, stride, 1], padding="VALID")
#         return conv

In [ ]:
# def deconv(batch_input, out_channels):
#     with tf.variable_scope("deconv"):
#         batch, in_height, in_width, in_channels = [int(d) for d in batch_input.get_shape()]
#         filter = tf.get_variable("filter", [4, 4, out_channels, in_channels], dtype=tf.float32,
#                                  initializer=tf.random_normal_initializer(0, 0.02))
#         # [batch, in_height, in_width, in_channels], [filter_width, filter_height, out_channels, in_channels]
#         #     => [batch, out_height, out_width, out_channels]
#         conv = tf.nn.conv2d_transpose(batch_input, filter, [batch, in_height * 2, in_width * 2, out_channels],
#                                       [1, 2, 2, 1], padding="SAME")
#         return conv